In [7]:
import pandas as pd

df = pd.read_csv("Results.csv")

df

,Text_ID,Project,Free_Text_KEY,Free_Text_title,Text,Is_latest_text
0,3,IDN0174,5,Project Objective,To recover the social and economic activity in...,0
1,12843,TUN1004,5,Project Objective,the objective is to address the urgent needs o...,1
2,12868,JOR1001,5,Project Objective,The objective of this project is to promote re...,1
3,12870,BHR1002,5,Project Objective,The objective of the project is to promote you...,1
4,12871,YEM1001,5,Project Objective,The objective of the Program is to contribute ...,1
...,...,...,...,...,...,...
27208,185127,PAK1060,459,Readiness,The project is part of the Work Program for 20...,1
27209,185129,MOZ1022,5,Project Objective,To organize cataract screening campaigns and p...,1
27210,185130,MOZ1022,7,Scope,"To perform screening 10,000 patients and Condu...",1
27211,185131,MOZ1022,6,Project Locations,Mozabique,1


In [8]:
selected_fields = ['Scope' , 'Project Relevance' , 'Project Context' , 'Project Objective' , 'Project Rationale']

#selected_fields = [ 'Project Objective' ]

filtered_df = df[df['Free_Text_title'].isin(selected_fields) & df['Is_latest_text'] == 1 ]

df2 = filtered_df[['Project' , 'Free_Text_title' , 'Text']].pivot(index = ['Project'] , columns='Free_Text_title' , values = 'Text').dropna()

df2.reset_index(inplace=True)

df2.head(5)

Free_Text_title,Project,Project Context,Project Objective,Project Rationale,Project Relevance,Scope
0,AFG1025,"UNICEF, the sole provider of ready-to-use ther...",The overall objective of this intervention is ...,"UNICEF, in technical coordination with partner...",Afghanistan is facing one of the world’s most ...,Procurement and distribution of ready-to-use t...
1,AFG1026,"UNICEF, the sole provider of ready-to-use ther...",The overall objective of this intervention is ...,"UNICEF, in technical coordination with partner...",Afghanistan is facing one of the world’s most ...,Procurement and distribution of ready-to-use t...
2,AFG1027,The Southern Region comprises the provinces of...,The overall goal of this pilot intervention is...,The Project will sustain essential services an...,The Project will sustain essential services an...,a. Provision of agricultural inputs such as ce...
3,AFG1030,"The limited access to WASH services, compounde...",Contribute to increased access to safe and sus...,"More vulnerable children, and their families i...",In urban areas the risk of AWD / cholera is pa...,- Provision of safe drinking water through cli...
4,AFG1031,"One of the health challenges in Afghanistan, i...","Provide essential Reproductive, Maternal, Neon...",The FHH model is also fully aligned with the p...,Afghanistan has made significant improvements ...,"This project is a collaboration between UNFPA,..."


In [9]:


df2 = df2.assign(Summary = 'Project:'+df2['Project'] + ' Objective:'+ df2['Project Objective'])



In [21]:
#pd.unique(df2.Free_Text_title)

In [10]:
df_in = df2[['Project' , 'Summary']].rename(columns={'Project' : 'source'})


#df_in = df2[['Project' , 'Summary']].rename(columns={'Project' : 'source'})

df_in.head(5)
#df_in = 

Free_Text_title,source,Summary
0,AFG1025,Project:AFG1025 Objective:The overall objectiv...
1,AFG1026,Project:AFG1026 Objective:The overall objectiv...
2,AFG1027,Project:AFG1027 Objective:The overall goal of ...
3,AFG1030,Project:AFG1030 Objective:Contribute to increa...
4,AFG1031,Project:AFG1031 Objective:Provide essential Re...


In [11]:
from langchain.document_loaders import DataFrameLoader


loader = DataFrameLoader(df_in, page_content_column="Summary")

docs = loader.load()

/Users/altaf/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
import os
import openai

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

#store = Chroma.from_documents(docs, embeddings)

store = Chroma.from_documents(docs, embeddings, collection_name='projects', persist_directory="./chroma_db2")
store.persist()

In [16]:
db = Chroma(persist_directory="./chroma_db2", embedding_function=embeddings, collection_name='projects' )

In [17]:
db.similarity_search("Which projects eleviate poverty?")

[Document(page_content='Project:AFG1037 Objective:Support vulnerable households in rural communities to have improved food security, household income, and resilience to climate change through the construction of agro-based infrastructure projects in the provinces of Badakhshan, Faryab, and Ghazni.', metadata={'source': 'AFG1037'}),
 Document(page_content='Project:BGD1087 Objective: Project Development Objective is as follows: i) to expand access to affordable finance for constructing 1,989 sustainable multi-storied housings with improved quality and necessary basic facilities for 70,632 lower and middle-income people dwelling in rural and peri-urban areas; ii) to ensure the optimum use of land and save cultivable land; and, iii) to address the climate change issue by introducing 55 climate-resilient housing for climatically vulnerable segments and 39 eco-friendly housing for reducing Green House Gases (GHG) emissions.', metadata={'source': 'BGD1087'}),
 Document(page_content='Project:G

In [18]:
store.similarity_search("Which projects eleviate poverty?")

[Document(page_content='Project:AFG1037 Objective:Support vulnerable households in rural communities to have improved food security, household income, and resilience to climate change through the construction of agro-based infrastructure projects in the provinces of Badakhshan, Faryab, and Ghazni.', metadata={'source': 'AFG1037'}),
 Document(page_content='Project:BGD1087 Objective: Project Development Objective is as follows: i) to expand access to affordable finance for constructing 1,989 sustainable multi-storied housings with improved quality and necessary basic facilities for 70,632 lower and middle-income people dwelling in rural and peri-urban areas; ii) to ensure the optimum use of land and save cultivable land; and, iii) to address the climate change issue by introducing 55 climate-resilient housing for climatically vulnerable segments and 39 eco-friendly housing for reducing Green House Gases (GHG) emissions.', metadata={'source': 'BGD1087'}),
 Document(page_content='Project:G

In [19]:
#from langchain.llms import OpenAIChat
from langchain.chat_models import ChatOpenAI

from langchain.chains import RetrievalQA

retriever = store.as_retriever()

#llm = OpenAIChat(temperature = 0.0, model_name="gpt-3.5-turbo-16k-0613")

llm = ChatOpenAI(temperature = 0.0, model_name="gpt-3.5-turbo-16k-0613")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [22]:
query = "Which projects eleviate poverty? And in which countries"

res = qa_chain.run(query)

print(res)



> Entering new RetrievalQA chain...

> Finished chain.
The projects that aim to alleviate poverty are:

1. Project: AFG1037 - This project supports vulnerable households in rural communities in Afghanistan, specifically in the provinces of Badakhshan, Faryab, and Ghazni.

2. Project: GIN1038 - This project focuses on reducing poverty and improving living conditions in Guinea, particularly in the prefectures of Faranah, Kankan, Labé, and Mamou.

3. Project: AFG1027 - This pilot intervention aims to support vulnerable livelihoods and reduce food insecurity in southern Afghanistan.

4. Project: BGD1087 - This project in Bangladesh aims to expand access to affordable finance for constructing sustainable multi-storied housings, addressing climate change issues and providing housing for vulnerable segments.

These projects are implemented in Afghanistan, Guinea, and Bangladesh respectively.


In [23]:
from IPython.display import display, Markdown
display(Markdown(res.replace('.' ,'.\n\n')))


The projects that aim to alleviate poverty are:

1.

 Project: AFG1037 - This project supports vulnerable households in rural communities in Afghanistan, specifically in the provinces of Badakhshan, Faryab, and Ghazni.



2.

 Project: GIN1038 - This project focuses on reducing poverty and improving living conditions in Guinea, particularly in the prefectures of Faranah, Kankan, Labé, and Mamou.



3.

 Project: AFG1027 - This pilot intervention aims to support vulnerable livelihoods and reduce food insecurity in southern Afghanistan.



4.

 Project: BGD1087 - This project in Bangladesh aims to expand access to affordable finance for constructing sustainable multi-storied housings, addressing climate change issues and providing housing for vulnerable segments.



These projects are implemented in Afghanistan, Guinea, and Bangladesh respectively.



In [75]:
from langchain.chains import RetrievalQAWithSourcesChain

chain_ws = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

query = "Provide a short summary on the projects related to housing"

res = chain_ws("question: Provide a short summary on the projects related to housing")

print(res)


{'question': 'question: Provide a short summary on the projects related to housing', 'answer': 'There are several projects related to housing:\n\n1. Project in Bangladesh: The objective of this project is to expand access to affordable finance for constructing sustainable multi-storied housings with improved quality and necessary basic facilities for lower and middle-income people dwelling in rural and peri-urban areas. It also aims to address climate change by introducing climate-resilient and eco-friendly housing.\n\n2. Project in Uzbekistan: This project aims to improve access to quality education by improving the learning environment, including the construction and equipment of new schools. It also focuses on improving the quality and efficiency of education services and the quality of pre-school education.\n\n3. Project in Afghanistan: The goal of this project is to support vulnerable households in rural communities to have improved food security, household income, and resilience 

In [79]:

display(Markdown(res.replace('.' ,'.\n\n')))
display(res)


{'question': 'question: Provide a short summary on the projects related to housing',
 'answer': 'There are several projects related to housing:\n\n1. Project in Bangladesh: The objective of this project is to expand access to affordable finance for constructing sustainable multi-storied housings with improved quality and necessary basic facilities for lower and middle-income people dwelling in rural and peri-urban areas. It also aims to address climate change by introducing climate-resilient and eco-friendly housing.\n\n2. Project in Uzbekistan: This project aims to improve access to quality education by improving the learning environment, including the construction and equipment of new schools. It also focuses on improving the quality and efficiency of education services and the quality of pre-school education.\n\n3. Project in Afghanistan: The goal of this project is to support vulnerable households in rural communities to have improved food security, household income, and resilience

In [89]:
t = df_in[ df_in['source'] == 'BGD1087' ]

## Chain with Custom Prompt

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer clearly, using internation development related jargon."""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [ ]:
res = query = "Provide a short summary on the projects related to housing"
qa.run(query)

display(Markdown(res.replace('.' ,'.\n\n')))